#  Konya Yurtlar İçin Yemek Dağıtım Problemi  
## Ant Colony Optimization (ACO) ile En Kısa Rota Bulma

**Öğrenci Adı:** Shams AL HAJJI  
**Öğrenci No:** 2112721301  

**GitHub Repo:**  
https://github.com/ShamsALHAJJI44/Konya_YemekDagitimi_ACO.git



## 📌 Problem Tanımı

Bu çalışmada, Konya ilinde bulunan öğrenci yurtlarına yemek dağıtımı yapan
bir aracın, tüm yurtları ziyaret edecek şekilde en kısa toplam mesafeyi
katetmesi amaçlanmıştır.

Problem, Gezgin Satıcı Problemi (TSP) kapsamında ele alınmış ve
Ant Colony Optimization (ACO) algoritması ile çözülmüştür.


## 🔍 Kullanılan Yöntem

Çözüm süreci aşağıdaki adımlardan oluşmaktadır:
1. Proje klasör yapısının oluşturulması
2. Yurt koordinatlarının CSV dosyasından okunması
3. Noktalar arası mesafe matrisinin oluşturulması
4. Ant Colony Optimization algoritmasının uygulanması
5. Sonuçların görselleştirilmesi ve kaydedilmesi


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


Bu adımda proje için gerekli klasör yapısı oluşturulmuştur.


In [ ]:
import os

ana_klasor = "/content/drive/MyDrive/Colab_Notebooks/Konya_YemekDagitimi_ACO"

klasorler = [
    "core",
    "data",
    "outputs"
]

# Alt klasörleri oluştur
for klasor in klasorler:
    os.makedirs(os.path.join(ana_klasor, klasor), exist_ok=True)

print("📁 Klasör yapısı oluşturuldu.")


📁 Klasör yapısı oluşturuldu.


## 📊 Veri Seti

Projede Konya ilinde bulunan 20 öğrenci yurdunun gerçek koordinatları
kullanılmıştır.

Veriler `konya_yurtlar.csv` dosyasında tutulmaktadır.
Her satırda:
- Yurt adı
- Enlem (Latitude)
- Boylam (Longitude)


In [ ]:
coordinates_code = '''
import pandas as pd
import os

# CSV dosyasından yurt koordinatlarını oku
def load_yurt_coordinates(csv_path):
    df = pd.read_csv(csv_path)
    sehir_koordinatlari = {}
    for i, row in df.iterrows():
        sehir_koordinatlari[i] = (row["name"], (row["lat"], row["lng"]))
    return sehir_koordinatlari

# Örnek kullanım:
# data_path = os.path.join(ana_klasor, "data", "konya_yurtlar.csv")
# sehir_koordinatlari = load_yurt_coordinates(data_path)
'''

with open(os.path.join(ana_klasor, "data", "coordinates.py"), "w") as f:
    f.write(coordinates_code.strip())

print("✅ data/coordinates.py dosyası oluşturuldu.")

##  Mesafe Hesabı

Noktalar arasındaki mesafelerin hesaplanmasında iki yöntem bulunmaktadır:
- Haversine mesafesi (doğrusal)
- OpenRouteService API ile gerçek sürüş mesafesi

Bu projede OpenRouteService Matrix API kullanılmıştır.


In [ ]:
haversine_code = '''
import math

def haversine(lat1, lon1, lat2, lon2):
    """
    Enlem ve boylam cinsinden verilen iki nokta arasındaki mesafeyi kilometre olarak döndürür.
    """
    R = 6371  # Dünya yarıçapı (km)
    phi1 = math.radians(lat1)
    phi2 = math.radians(lat2)
    dphi = math.radians(lat2 - lat1)
    dlambda = math.radians(lon2 - lon1)

    a = math.sin(dphi / 2)**2 + math.cos(phi1) * math.cos(phi2) * math.sin(dlambda / 2)**2
    c = 2 * math.asin(math.sqrt(a))
    return R * c
'''

with open(os.path.join(ana_klasor, "core", "haversine.py"), "w") as f:
    f.write(haversine_code.strip())

print("✅ core/haversine.py dosyası oluşturuldu.")


✅ core/haversine.py dosyası oluşturuldu.


###  Mesafe Matrisi Oluşturma


In [ ]:
matrix_utils_code = '''
import numpy as np
from core.haversine import haversine

def mesafe_matrisi_olustur(koordinatlar):
    n = len(koordinatlar)
    mesafe = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            if i == j:
                mesafe[i][j] = np.inf
            else:
                lat1, lon1 = koordinatlar[i]
                lat2, lon2 = koordinatlar[j]
                mesafe[i][j] = haversine(lat1, lon1, lat2, lon2)
    return mesafe

def hesapla_cekicilik(mesafe):
    cekicilik = np.zeros_like(mesafe)
    with np.errstate(divide='ignore'):
        cekicilik = 1 / mesafe
        cekicilik[mesafe == np.inf] = 0
    return cekicilik
'''

with open(os.path.join(ana_klasor, "core", "matrix_utils.py"), "w") as f:
    f.write(matrix_utils_code.strip())

print("✅ core/matrix_utils.py dosyası oluşturuldu.")


✅ core/matrix_utils.py dosyası oluşturuldu.


## Ant Colony Optimization (ACO)

ACO algoritması, karıncaların feromon izlerini takip ederek
en kısa yolu bulma prensibine dayanmaktadır.


In [ ]:
ant_algorithm_code = '''
import numpy as np
import random
from core.matrix_utils import hesapla_cekicilik

def olasilik_hesapla(mevcut, ziyaret_edilmemisler, feromon, cekicilik, alpha, beta):
    toplam = 0
    olasiliklar = {}
    for j in ziyaret_edilmemisler:
        deger = (feromon[mevcut][j] ** alpha) * (cekicilik[mevcut][j] ** beta)
        olasiliklar[j] = deger
        toplam += deger
    for j in olasiliklar:
        olasiliklar[j] /= toplam if toplam > 0 else 1
    return olasiliklar

def rulet_tekerlegi_secimi(olasilik_dict):
    r = random.random()
    toplam = 0
    for sehir, olasilik in olasilik_dict.items():
        toplam += olasilik
        if r <= toplam:
            return sehir

def karinca_gezi(baslangic, mesafe, feromon, alpha=1, beta=2):
    n = len(mesafe)
    yol = [baslangic]
    toplam_uzunluk = 0
    cekicilik = hesapla_cekicilik(mesafe)
    while len(yol) < n:
        mevcut = yol[-1]
        ziyaret_edilmemisler = list(set(range(n)) - set(yol))
        olasiliklar = olasilik_hesapla(mevcut, ziyaret_edilmemisler, feromon, cekicilik, alpha, beta)
        secilen = rulet_tekerlegi_secimi(olasiliklar)
        yol.append(secilen)
        toplam_uzunluk += mesafe[mevcut][secilen]
    toplam_uzunluk += mesafe[yol[-1]][yol[0]]  # geri dönüş mesafesi
    yol.append(yol[0])  # başlangıç noktasına dön
    return yol, toplam_uzunluk

def feromon_guncelle(feromon, yollar, buharlasma_orani=0.5, Q=1.0):
    yeni_feromon = (1 - buharlasma_orani) * feromon
    for yol, uzunluk in yollar:
        for i in range(len(yol) - 1):
            a, b = yol[i], yol[i + 1]
            katkı = Q / (uzunluk + 1e-6)  # sıfıra bölünme hatasını önlemek için
            yeni_feromon[a][b] += katkı
            yeni_feromon[b][a] += katkı
    return yeni_feromon

def run_aco(mesafe, karinca_sayisi=10, iterasyon_sayisi=50, alpha=1, beta=3,
            buharlasma_orani=0.5, feromon_katkisi=1):
    sehir_sayisi = len(mesafe)
    feromon = np.ones_like(mesafe) * 0.1
    en_iyi_yol = None
    en_kisa_mesafe = float("inf")
    iterasyon_en_iyiler = []
    for it in range(iterasyon_sayisi):
        yollar = []
        for _ in range(karinca_sayisi):
            yol, uzunluk = karinca_gezi(0, mesafe, feromon, alpha, beta)
            yollar.append((yol, uzunluk))
            if uzunluk < en_kisa_mesafe:
                en_kisa_mesafe = uzunluk
                en_iyi_yol = yol
        feromon = feromon_guncelle(feromon, yollar, buharlasma_orani, feromon_katkisi)
        iterasyon_en_iyiler.append(en_kisa_mesafe)
        print(f"🔁 İterasyon {it+1}: En kısa mesafe = {en_kisa_mesafe:.2f} km")
    return en_iyi_yol, en_kisa_mesafe, iterasyon_en_iyiler
'''

with open(os.path.join(ana_klasor, "core", "ant_algorithm.py"), "w") as f:
    f.write(ant_algorithm_code.strip())

print("✅ core/ant_algorithm.py dosyası oluşturuldu.")


✅ core/ant_algorithm.py dosyası oluşturuldu.


In [ ]:
plotting_code = '''
import matplotlib.pyplot as plt

def yol_gorsellestir(yol, koordinatlar, kaydet=False, dosya_yolu=None):
    plt.figure(figsize=(10, 8))
    for i in range(len(yol) - 1):
        sehir1, (lat1, lon1) = koordinatlar[yol[i]]
        sehir2, (lat2, lon2) = koordinatlar[yol[i + 1]]
        plt.plot([lon1, lon2], [lat1, lat2], 'k--', alpha=0.6)
        plt.scatter(lon1, lat1, color='red', s=40)
        plt.text(lon1 + 0.01, lat1 + 0.01, sehir1, fontsize=9, color='blue')
    son_sehir, (son_lat, son_lon) = koordinatlar[yol[-1]]
    plt.scatter(son_lon, son_lat, color='red', s=40)
    plt.text(son_lon + 0.01, son_lat + 0.01, son_sehir, fontsize=9, color='blue')
    plt.title("Konya Yurt Dağıtım Rotası")
    plt.xlabel("Boylam")
    plt.ylabel("Enlem")
    plt.grid(True)

    if kaydet and dosya_yolu:
        plt.savefig(dosya_yolu, dpi=300)
        print(f"✅ Yol grafiği kaydedildi: {dosya_yolu}")
    else:
        plt.show()

def plot_convergence(best_distances, kaydet=False, dosya_yolu=None):
    plt.figure(figsize=(8, 6))
    plt.plot(best_distances, marker='o', color='green')
    plt.title("ACO - En İyi Mesafenin İterasyonlara Göre Değişimi")
    plt.xlabel("İterasyon")
    plt.ylabel("En Kısa Mesafe (km)")
    plt.grid(True)

    if kaydet and dosya_yolu:
        plt.savefig(dosya_yolu, dpi=300)
        print(f"✅ Yakınsama grafiği kaydedildi: {dosya_yolu}")
    else:
        plt.show()
'''

with open(os.path.join(ana_klasor, "outputs", "plotting.py"), "w") as f:
    f.write(plotting_code.strip())

print("✅ visual/plotting.py dosyası oluşturuldu.")


### ⚙️ Algoritma Parametreleri


In [ ]:
with open("/content/drive/MyDrive/Colab_Notebooks/Konya_YemekDagitimi_ACO/config.py", "w") as f:
    f.write("""
# ACO parametreleri burada tutulur

ACO_CONFIG = {
    "karinca_sayisi": 10,
    "iterasyon_sayisi": 30,
    "alpha": 1.0,
    "beta": 2.0,
    "buharlasma_orani": 0.5,
    "feromon_katkisi": 1.0
}
    """.strip())


In [ ]:
##API Key Tanımı
import os
os.environ["ORS_API_KEY"] = "YOUR_API_KEY_HERE"


## ▶ Algoritmanın Çalıştırılması


In [ ]:
with open("/content/drive/MyDrive/Colab_Notebooks/Konya_YemekDagitimi_ACO/main.py", "w") as f:
    f.write("""
import os
from data.coordinates import load_yurt_coordinates
from core.matrix_utils import mesafe_matrisi_olustur
from core.ant_algorithm import run_aco
from config import ACO_CONFIG
from outputs.plotting import yol_gorsellestir, plot_convergence
import numpy as np

# Veriyi yükle
data_path = os.path.join("/content/drive/MyDrive/Colab_Notebooks/Konya_YemekDagitimi_ACO/data", "konya_yurtlar.csv")
sehir_koordinatlari = load_yurt_coordinates(data_path)

# Sadece koordinatları listeye dönüştür
koordinatlar = [v[1] for v in sehir_koordinatlari.values()]

# Mesafe matrisini oluştur
mesafe = mesafe_matrisi_olustur(koordinatlar)

# ACO algoritmasını çalıştır
en_iyi_yol, en_iyi_mesafe, iterasyon_iyileri = run_aco(
    mesafe,
    karinca_sayisi=ACO_CONFIG["karinca_sayisi"],
    iterasyon_sayisi=ACO_CONFIG["iterasyon_sayisi"],
    alpha=ACO_CONFIG["alpha"],
    beta=ACO_CONFIG["beta"],
    buharlasma_orani=ACO_CONFIG["buharlasma_orani"],
    feromon_katkisi=ACO_CONFIG["feromon_katkisi"]
)

# Sonuçları yazdır
print("\\n🚚 En iyi rota:")
for i in en_iyi_yol:
    print(f"  ➤ {sehir_koordinatlari[i][0]}")
print(f"\\n📏 Toplam mesafe: {en_iyi_mesafe:.2f} km")

# Sonuç dosyasına kaydet
output_dir = "/content/drive/MyDrive/Colab_Notebooks/Konya_YemekDagitimi_ACO/outputs"
os.makedirs(output_dir, exist_ok=True)

# .txt dosyasına kaydet
with open(os.path.join(output_dir, "sonuc.txt"), "w") as out:
    out.write(f"En iyi yol: {en_iyi_yol}\\nToplam mesafe: {en_iyi_mesafe:.2f} km\\n")

# Görselleri kaydet
rota_dosya = os.path.join(output_dir, "rota.png")
yakinsama_dosya = os.path.join(output_dir, "yakinsama.png")

yol_gorsellestir(en_iyi_yol, sehir_koordinatlari, kaydet=True, dosya_yolu=rota_dosya)
plot_convergence(iterasyon_iyileri, kaydet=True, dosya_yolu=yakinsama_dosya)

print("\\n✅ Çıktılar kaydedildi:")
print(f"📁 Rota grafiği: {rota_dosya}")
print(f"📉 Yakınsama grafiği: {yakinsama_dosya}")
print(f"🧾 Sonuç dosyası: {os.path.join(output_dir, 'sonuc.txt')}")
""")


In [ ]:
!python /content/drive/MyDrive/Colab_Notebooks/Konya_YemekDagitimi_ACO/main.py


🔁 İterasyon 1: En kısa mesafe = 68.14 km
🔁 İterasyon 2: En kısa mesafe = 68.14 km
🔁 İterasyon 3: En kısa mesafe = 61.79 km
🔁 İterasyon 4: En kısa mesafe = 61.79 km
🔁 İterasyon 5: En kısa mesafe = 58.99 km
🔁 İterasyon 6: En kısa mesafe = 58.99 km
🔁 İterasyon 7: En kısa mesafe = 58.99 km
🔁 İterasyon 8: En kısa mesafe = 58.99 km
🔁 İterasyon 9: En kısa mesafe = 58.99 km
🔁 İterasyon 10: En kısa mesafe = 58.99 km
🔁 İterasyon 11: En kısa mesafe = 58.99 km
🔁 İterasyon 12: En kısa mesafe = 58.99 km
🔁 İterasyon 13: En kısa mesafe = 58.99 km
🔁 İterasyon 14: En kısa mesafe = 58.99 km
🔁 İterasyon 15: En kısa mesafe = 58.99 km
🔁 İterasyon 16: En kısa mesafe = 58.99 km
🔁 İterasyon 17: En kısa mesafe = 58.99 km
🔁 İterasyon 18: En kısa mesafe = 58.99 km
🔁 İterasyon 19: En kısa mesafe = 58.99 km
🔁 İterasyon 20: En kısa mesafe = 58.99 km
🔁 İterasyon 21: En kısa mesafe = 58.99 km
🔁 İterasyon 22: En kısa mesafe = 58.99 km
🔁 İterasyon 23: En kısa mesafe = 58.99 km
🔁 İterasyon 24: En kısa mesafe = 58.99 km
🔁

In [ ]:
!ls /content/drive/MyDrive/Colab_Notebooks/Konya_YemekDagitimi_ACO/outputs


plotting.py  __pycache__  rota.png  sonuc.txt  yakinsama.png


##  Sonuçlar

- En kısa rota başarıyla bulunmuştur
- Toplam mesafe kilometre cinsinden hesaplanmıştır
- Yakınsama grafiği algoritmanın performansını göstermektedir


##  Genel Değerlendirme

Bu çalışmada gerçek veriler ve gerçek yol mesafeleri kullanılarak
yemek dağıtım problemi çözülmüştür.
Ant Colony Optimization (Karınca) algoritmasının bu tür optimizasyon
problemlerinde etkili olduğu gözlemlenmiştir.
